# Introduction

QA plots for the generic distortions reconstruction performance

In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

In [2]:
# turn off/on code for the result HTML page

display(Markdown('*For the result HTML page:* '))
    
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 $('div.jp-CodeMirrorEditor').hide(); // for newer versions of nbviewer.org
 } else {
 $('div.input').show();
 $('div.jp-CodeMirrorEditor').show(); // for newer versions of nbviewer.org
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

*For the result HTML page:* 

In [3]:
import os.path

# readme file of the macros, available if run under JenkinsCI
# https://github.com/sPHENIX-Collaboration/utilities/blob/master/jenkins/built-test/test-tracking-qa.sh 

macro_markdown = 'Fun4All-macros-README.md'


if os.path.isfile(macro_markdown) :
    with open(macro_markdown, 'r') as file:
        display(Markdown(file.read()))

[//]: # (This section starts with a 2nd level heading and get embedded in the result QA report at https://github.com/sPHENIX-Collaboration/QA-gallery/tree/QA-tracking-low-occupancy)

## Tracking Distortions

Test tracking QA distortions

The source code of the macro can be found at https://github.com/sPHENIX-Collaboration/macros/tree/QA-tracking-distortions or [comparing it to the master branch](https://github.com/sPHENIX-Collaboration/macros/compare/QA-tracking-distortions?expand=1).


## `pyROOT` env check

In [4]:
import ROOT

OFFLINE_MAIN = os.getenv("OFFLINE_MAIN")
if OFFLINE_MAIN is not None:
    display(Markdown(f"via sPHENIX software distribution at `{OFFLINE_MAIN}`"))

Welcome to JupyROOT 6.26/06


via sPHENIX software distribution at `/var/lib/jenkins/workspace/sPHENIX/Build-Master-gcc12@6/build/new/install.1`

## Plotting source code

In [5]:
import subprocess

try:
    git_url = \
        subprocess.run(['git','remote','get-url','origin'], stdout=subprocess.PIPE)\
        .stdout.decode('utf-8').strip()\
        .replace('git@github.com:','https://github.com/')

    display(Markdown(f"View the source code repository at {git_url}"))
except: # catch *all* exceptions
    #     well do nothing
    pass

View the source code repository at https://github.com/sPHENIX-Collaboration/QA-gallery.git

In [6]:
display(Markdown('Some further details about the QA run, if executed under the Jenkins CI:'))

checkrun_repo_commit = os.getenv("checkrun_repo_commit")
if checkrun_repo_commit is not None:
    display(Markdown(f"* The commit being checked is {checkrun_repo_commit}"))
        
ghprbPullLink = os.getenv("ghprbPullLink")
if ghprbPullLink is not None:
    display(Markdown(f"* Link to the pull request: {ghprbPullLink}"))

BUILD_URL =  os.getenv("BUILD_URL")
if BUILD_URL is not None:
    display(Markdown(f"* Link to the build: {BUILD_URL}"))

git_url_macros =  os.getenv("git_url_macros")
sha_macros =  os.getenv("sha_macros")
if git_url_macros is not None:
    display(Markdown(f"* Git repo for macros: {git_url_macros} , which merges `{sha_macros}` and the QA tracking branch"))

RUN_ARTIFACTS_DISPLAY_URL = os.getenv("RUN_ARTIFACTS_DISPLAY_URL")
if RUN_ARTIFACTS_DISPLAY_URL is not None:
    display(Markdown(f"* Download the QA ROOT files: {RUN_ARTIFACTS_DISPLAY_URL}"))

JENKINS_URL  = os.getenv("JENKINS_URL")
if JENKINS_URL is not None:
    display(Markdown(f"Automatically generated by [sPHENIX Jenkins continuous integration]({JENKINS_URL}) [![sPHENIX](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/sphenix-logo-white-bg-72p.png)](https://www.sphenix.bnl.gov/web/) &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; [![jenkins.io](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/jenkins_logo_title-72p.png)](https://jenkins.io/)"))

Some further details about the QA run, if executed under the Jenkins CI:

* The commit being checked is sPHENIX-Collaboration/coresoftware/4951596025632eb3d210ab5bac71dd3a990ec16d

* Link to the pull request: https://github.com/sPHENIX-Collaboration/coresoftware/pull/2614

* Link to the build: https://web.sdcc.bnl.gov/jenkins-sphenix/job/sPHENIX/job/test-tracking-distortions-qa/1411/

* Git repo for macros: https://github.com/sPHENIX-Collaboration/macros.git , which merges `master` and the QA tracking branch

* Download the QA ROOT files: https://web.sdcc.bnl.gov/jenkins-sphenix/job/sPHENIX/job/test-tracking-distortions-qa/1411/display/redirect?page=artifacts

Automatically generated by [sPHENIX Jenkins continuous integration](https://web.sdcc.bnl.gov/jenkins-sphenix/) [![sPHENIX](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/sphenix-logo-white-bg-72p.png)](https://www.sphenix.bnl.gov/web/) &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; [![jenkins.io](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/jenkins_logo_title-72p.png)](https://jenkins.io/)


# Initialization

In [7]:
%%cpp -d

#include "QA_Draw_Utility.C"

#include <sPhenixStyle.C>

#include <TFile.h>
#include <TLine.h>
#include <TString.h>
#include <TTree.h>
#include <cassert>
#include <cmath>

In [8]:
%%cpp

SetsPhenixStyle();
TVirtualFitter::SetDefaultFitter("Minuit2");

sPhenixStyle: Applying nominal settings.
sPhenixStyle: ROOT6 mode


In [9]:
%jsroot on

## Inputs and file checks

In [10]:

qa_file_name_new = os.getenv("qa_file_name_new")
if qa_file_name_new is None:
    qa_file_name_new = "G4sPHENIX_test-tracking_Event500_Sum5_qa.root"
    display(Markdown(f"`qa_file_name_new` env not set. use the default `qa_file_name_new={qa_file_name_new}`"))

qa_file_name_ref = os.getenv("qa_file_name_ref")
if qa_file_name_ref is None:
    qa_file_name_ref = qa_file_name_new
    display(Markdown(f"`qa_file_name_ref` env not set. use the default same as new file `qa_file_name_ref={qa_file_name_ref}`"))
elif qa_file_name_ref == 'None':
    qa_file_name_ref = qa_file_name_new
    display(Markdown(f"`qa_file_name_ref` env not set. use the default same as new file `qa_file_name_ref={qa_file_name_ref}`"))
    


In [11]:
# qa_file_new = ROOT.TFile.Open(qa_file_name_new);

# assert qa_file_new.IsOpen()
# qa_file_new.ls()
display(Markdown(f"Opening QA file at `{qa_file_name_new}`"))
ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_new = new TFile(\"{qa_file_name_new}\");")
ROOT.gInterpreter.ProcessLine(f"const char * qa_file_name_new = \"{qa_file_name_new}\";")

if qa_file_name_ref is not None:
#     qa_file_ref = ROOT.TFile.Open(qa_file_name_ref);

#     assert qa_file_ref.IsOpen()
    display(Markdown(f"Opening QA reference file at `{qa_file_name_ref}`"))
    ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_ref = new TFile(\"{qa_file_name_ref}\");")
    ROOT.gInterpreter.ProcessLine(f"const char *  qa_file_name_ref = \"{qa_file_name_ref}\";")
else:
    ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_ref = nullptr;")
    ROOT.gInterpreter.ProcessLine(f"const char *  qa_file_name_ref = nullptr;")


Opening QA file at `G4sPHENIX_test-tracking_Event700_Sum3_qa.root`

Opening QA reference file at `reference/G4sPHENIX_test-tracking_Event700_Sum3_qa.root`

In [12]:
%%cpp

if (qa_file_new == nullptr) 
{
    cout <<"Error, can not open QA root file"<<qa_file_name_new<<endl;
    exit(1);
}

if (qa_file_ref == nullptr) 
{
    cout <<"Error, can not open ref QA root file"<<qa_file_name_ref<<endl;
    exit(1);
}

if (not qa_file_new->IsOpen()) 
{
    cout <<"Error, can not open QA root file"<<qa_file_name_new<<endl;
    exit(1);
}

if (not qa_file_ref->IsOpen()) 
{
    cout <<"Error, can not open ref QA root file"<<qa_file_name_ref<<endl;
    exit(1);
}

# Delta Rphi residuals (track - cluster) vs layer number

In [13]:
%%cpp
    const char *hist_name_prefix = "QAG4SimulationDistortions";
    TString prefix = TString("h_") + hist_name_prefix + TString("_");
    
    TCanvas *cv_deltarphi_layer = new TCanvas(TString("QA_Draw_rphi_residuals") + TString("_") + hist_name_prefix,
                                     TString("QA_Draw_rphi_residuals") + TString("_") + hist_name_prefix,
                                     1200,500);
    cv_deltarphi_layer->Divide(2, 1);
        
    auto h_deltarphi_layer_ref = (TH2F *)qa_file_ref->GetObjectChecked(prefix + "deltarphi_layer", "TH2");
    if( !h_deltarphi_layer_ref )
    { 
        std::cout << "h_deltarphi_layer_ref not found. doing nothing." << std::endl;
        return;
    }
    h_deltarphi_layer_ref->SetDirectory(nullptr);

    auto h_deltarphi_layer_new = (TH2F *)qa_file_new->GetObjectChecked(prefix + "deltarphi_layer", "TH2");
    if( !h_deltarphi_layer_new )
    { 
        std::cout << "h_deltarphi_layer_new not found. doing nothing." << std::endl;
        return;
    }
    
    {
        auto newScale = FitResolution_hist(h_deltarphi_layer_new, false, 1);
        newScale->GetXaxis()->SetTitle("layer");
        newScale->GetYaxis()->SetTitle("#LT#Delta r#phi#GT (#mum)");
        newScale->Scale(1e4);
        newScale->SetMinimum(-100);
        newScale->SetMaximum(100);

        auto refScale = FitResolution_hist(h_deltarphi_layer_ref, false, 1);
        refScale->Scale(1e4);

        cv_deltarphi_layer->cd(1);
        DrawReference(newScale, refScale);
        
        // Draw line at zero
        auto line = HorizontalLine( gPad, 0 );
        line->Draw();
    }
    
    {
        auto newRMS = FitResolution_hist(h_deltarphi_layer_new, false, 2);
        newRMS->GetXaxis()->SetTitle("layer");
        newRMS->GetYaxis()->SetTitle("RMS(#Delta r#phi) (#mum)");
        newRMS->Scale(1e4);
        newRMS->SetMinimum(0);
        newRMS->SetMaximum(1200);
        
        auto refRMS = FitResolution_hist(h_deltarphi_layer_ref, false, 2);
        refRMS->Scale(1e4);
        
        cv_deltarphi_layer->cd(2);
        DrawReference(newRMS, refRMS);
    }
    
    
    cv_deltarphi_layer->Draw();
    

Warning in <TFile::Append>: Replacing existing TH1: h_QAG4SimulationDistortions_deltarphi_layer_FitResolution (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_QAG4SimulationDistortions_deltarphi_layer_FitResolution (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_QAG4SimulationDistortions_deltarphi_layer_FitResolution (Potential memory leak).


# Delta z residuals (track - cluster) vs layer number

In [14]:
%%cpp
    const char *hist_name_prefix = "QAG4SimulationDistortions";
    TString prefix = TString("h_") + hist_name_prefix + TString("_");
    
    TCanvas *cv_deltaz_layer = new TCanvas(TString("QA_Draw_z_residuals") + TString("_") + hist_name_prefix,
                                     TString("QA_Draw_z_residuals") + TString("_") + hist_name_prefix,
                                     1200,500);
    cv_deltaz_layer->Divide(2, 1);
        
    auto h_deltaz_layer_ref = (TH2F *)qa_file_ref->GetObjectChecked(prefix + "deltaz_layer", "TH2");
    if( !h_deltaz_layer_ref )
    { 
        std::cout << "h_deltaz_layer_ref not found. doing nothing." << std::endl;
        return;
    }
    h_deltaz_layer_ref->SetDirectory(nullptr);

    auto h_deltaz_layer_new = (TH2F *)qa_file_new->GetObjectChecked(prefix + "deltaz_layer", "TH2");
    if( !h_deltaz_layer_new )
    { 
        std::cout << "h_deltaz_layer_new not found. doing nothing." << std::endl;
        return;
    }
    h_deltaz_layer_new->SetDirectory(nullptr);
    
    {
        auto newScale = FitResolution_hist(h_deltaz_layer_new, false, 1);
        newScale->GetXaxis()->SetTitle("layer");
        newScale->GetYaxis()->SetTitle("#LT#Delta z#GT (#mum)");
        newScale->Scale(1e4);
        newScale->SetMinimum(-100);
        newScale->SetMaximum(100);

        auto refScale = FitResolution_hist(h_deltaz_layer_ref, false, 1);
        refScale->Scale(1e4);

        cv_deltaz_layer->cd(1);
        DrawReference(newScale, refScale);
        
        // Draw line at zero
        auto line = HorizontalLine( gPad, 0 );
        line->Draw();
    }
    
    {
        auto newRMS = FitResolution_hist(h_deltaz_layer_new, false, 2);
        newRMS->GetXaxis()->SetTitle("layer");
        newRMS->GetYaxis()->SetTitle("RMS(#Delta z) (#mum)");
        newRMS->Scale(1e4);
        newRMS->SetMinimum(0);
        newRMS->SetMaximum(2000);
        
        auto refRMS = FitResolution_hist(h_deltaz_layer_ref, false, 2);
        refRMS->Scale(1e4);
        
        cv_deltaz_layer->cd(2);
        DrawReference(newRMS, refRMS);
    }
    
    cv_deltaz_layer->Draw();
    

Warning in <TFile::Append>: Replacing existing TH1: h_QAG4SimulationDistortions_deltaz_layer_FitResolution (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_QAG4SimulationDistortions_deltaz_layer_FitResolution (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_QAG4SimulationDistortions_deltaz_layer_FitResolution (Potential memory leak).


In [15]:
%%cpp

KSTestSummary::getInstance()->make_summary_txt("QA-Distortions.txt");

This notebook contains 4 KSTets: combined Chi2/nDoF = 0.0427052 / 8, and combined __p-Value = 1__


In [16]:
%%cpp

KSTestSummary::getInstance()->make_summary_TCanvas() -> Draw();